In [16]:
from random import *
import vectors
from math import *
import pygame
import numpy as np
from vectors import distance

In [2]:
class PolygonModel():
    def __init__(self,points):
        self.points=points
        self.rotation_angle=0
        self.x=0
        self.y=0
    def transformed(self):
        rotated = [vectors.rotate2d(self.rotation_angle, v) for v in self.points]
        return [vectors.add((self.x,self.y),v) for v in rotated]
    def does_intersect(self,segment):
        pass

In [3]:
class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5,0),(-0.25,0.25),(-0.25,-0.25)])

In [4]:
class Asteroids(PolygonModel):
    def __init__(self):
        sides=randint(5,9)
        vs=[vectors.to_cartesian((uniform(0.5,1.0),2*pi*i/sides))for i in range(0,sides)]
        super().__init__(vs)

In [5]:
ship=Ship()
asteroid_count=10
asteroids=[Asteroids() for _ in range(0,asteroid_count)]

for ast in asteroids:
    ast.x=randint(-9,9)
    ast.y=randint(-9,9)    

In [6]:
width,height=400,400
def to_pixels(x,y):
    return (width/2+width*x/20,height/2-height*y/20)

In [7]:
GREEN=(0,255,0)
def draw_poly(screen,polygon_model,color=GREEN):
    pixel_points=[to_pixels(x,y) for x,y in polygon_model.transformed()]
    pygame.draw.aalines(screen,color,True,pixel_points,10)

In [8]:
class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5,0),(-0.25,0.25),(-0.25,-0.25)])
    def laser_segment(self):
        dist=20.*sqrt(2)
        x,y=self.transformed()[0]
        return ((x,y),
               (x+dist*cos(self.rotation_angle),
               y+dist*sin(self.rotation_angle)))

In [9]:
RED=(255,0,0)
def draw_segment(screen, v1,v2,color=RED):
    pygame.draw.aaline(screen,color,to_pixels(*v1), to_pixels(*v2),10)

In [10]:
# ship=Ship()
# asteroid_count=10
# asteroids=[Asteroids() for _ in range(0,asteroid_count)]

# for ast in asteroids:
#     ast.x=randint(-9,9)
#     ast.y=randint(-9,9)    

# laser=ship.laser_segment()
# keys=pygame.key.get_pressed()
# if keys[pygame.K_SPACE]:
#     draw_segment(*laser)
# for asteroid in asteroids:
#     if asteroid.does_intersect(segment):
#         asteroids.remove(asteroid)

In [12]:
matrix=np.array(((-1,1),(1,2)))
output=np.array((0,8))

In [13]:
np.linalg.solve(matrix,output)

array([2.66666667, 2.66666667])

In [14]:
def standard_form(v1,v2):
    x1,y1=v1
    x2,y2=v2    
    a=y2-y1
    b=x1-x2
    c=x1*y2-x2*y1
    return a,b,c

In [15]:
def intersection(u1,u2,v1,v2):
    a1,b1,c1=standard_form(u1,u2)
    a2,b2,c2=standard_form(v1,v2)    
    m=np.array(((a1,b1),(a2,b2)))
    c=np.array((c1,c2))
    return np.linalg.solve(m,c)

In [17]:
def do_segments_intersect(s1,s2):
    u1,u2=s1
    v1,v2=s2
    d1,d2=distance(*s1),distance(*s2)
    x,y=intersection(u1,u2,v1,v2)
    return (distance(u1,(x,y))<=d1 and
               distance(u2,(x,y))<=d1 and
               distance(v1,(x,y))<=d2 and
               distance(v2,(x,y))<=d2)

In [18]:
class PolygonModel():
    def __init__(self,points):
        self.points=points
        self.rotation_angle=0
        self.x=0
        self.y=0
        
    def transformed(self):
        rotated = [vectors.rotate2d(self.rotation_angle, v) for v in self.points]
        return [vectors.add((self.x,self.y),v) for v in rotated]
    
    def segments(self):
        point_count=len(self.points)
        points=self.transformed()
        return [(points[i],points[(i+1)%point_count])
               for i in range(0,point_count)]
    
    def does_intersect(self,other_segment):
        for segment in self.segments():
            if do_segments_intersect(other_segment,segment):
                return True
        return False